# Import




In [ ]:
#!pip install camel_tools
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
from __future__ import unicode_literals
#from tashaphyne.stemming import ArabicLightStemmer
from camel_tools.utils.normalize import normalize_alef_maksura_ar
from camel_tools.utils.normalize import normalize_alef_ar
from camel_tools.utils.normalize import normalize_teh_marbuta_ar
from camel_tools.utils.dediac import dediac_ar


In [ ]:

import torch
from torch.jit import script, trace
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
import csv
import random
import re
import os
import unicodedata
import codecs
from io import open
import itertools
import math
import pandas as pd
from tqdm import tqdm
import numpy as np
from sounds.speak_ import  speak,speak_ar
from sounds.azure_speech import speech2txt

#ArListem = ArabicLightStemmer()

USE_CUDA = torch.cuda.is_available()
device = torch.device("cuda" if USE_CUDA else "cpu")

# Vocab

In [ ]:
# Default word tokens
PAD_token = 0  # Used for padding short sentences
SOS_token = 1  # Start-of-sentence token
EOS_token = 2  # End-of-sentence token

class Voc:
    def __init__(self, name):
        self.name = name
        self.trimmed = False
        self.word2index = {}
        self.word2count = {}
        self.index2word = {PAD_token: "PAD", SOS_token: "SOS", EOS_token: "EOS"}
        self.num_words = 3  # Count SOS, EOS, PAD

    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.num_words
            self.word2count[word] = 1
            self.index2word[self.num_words] = word
            self.num_words += 1
        else:
            self.word2count[word] += 1

    # Remove words below a certain count threshold
    def trim(self, min_count):
        if self.trimmed:
            return
        self.trimmed = True

        keep_words = []

        for k, v in self.word2count.items():
            if v >= min_count:
                keep_words.append(k)

        # Reinitialize dictionaries
        self.word2index = {}
        self.word2count = {}
        self.index2word = {PAD_token: "PAD", SOS_token: "SOS", EOS_token: "EOS"}
        self.num_words = 3 # Count default tokens

        for word in keep_words:
            self.addWord(word)

# data load

### clean

In [ ]:
# Turn a Unicode string to plain ASCII
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )


def normalizeString(s):
    s = unicodeToAscii(s.lower().strip())
    s = re.sub(r"([.!?؟])", r" \1", s)
    s = re.sub(r"\s+", r" ", s).strip().replace("?","").replace("؟","").replace("!","")
    s = s.replace(".", "")
    return s

def normalize_egypt(txt):
    # Normalize alef variants to 'ا'
    txt = normalize_alef_ar(txt)
    # Normalize alef maksura 'ى' to yeh 'ي'
    txt = normalize_alef_maksura_ar(txt)
    # Normalize teh marbuta 'ة' to heh 'ه'
    txt = normalize_teh_marbuta_ar(txt)
    txt = dediac_ar(txt)
    #txt = txt.replace("و", " و ")
    return txt

In [ ]:
def loadPrepareData(corpus, corpus_name, datafile):
    # Read 
    pairs = sents_egypt_all
    voc = Voc(corpus_name)
    pairs = [pair for pair in pairs if len(pair[0].split(' ')) < MAX_LENGTH 
              and len(pair[1].split(' ')) < MAX_LENGTH]
    
    # word2index & index2word
    for pair in pairs:
        voc.addSentence(pair[0])
        voc.addSentence(pair[1])
    return voc, pairs


In [ ]:
normalize_egypt("هَل تَعلَمِي يَا مُنَى أَن دُكتُور الجَامِعَة طَلَب مِني أَن أَكتُب مَوضُوع عَن العَائِلَة العَرَبِيَة.")

# Data

In [ ]:
A_egypt = open("C:/Users/AnyOneElse/GP_POP/Batot/data/response.txt", encoding='utf-8'). read().strip().split('\n')
Q_egypt = open("C:/Users/AnyOneElse/GP_POP/Batot/data/Qs.txt", encoding='utf-8'). read().strip().split('\n')
sents_egypt = []
print(len(A_egypt))
print(len(Q_egypt))

for i in tqdm(range(len(A_egypt))):
    s = []
    s.append(Q_egypt[i])
    s.append(A_egypt[i])
    sents_egypt.append(s)
#sents_egypt

In [ ]:
for i in tqdm(range( len(sents_egypt))):
    q = sents_egypt[i][0]
    a = sents_egypt[i][1]
    ##########
    sents_egypt[i][0] = normalize_egypt(q)
    sents_egypt[i][1] = normalize_egypt(a)

In [ ]:
egypt_conv = open("C:/Users/AnyOneElse/GP_POP/Batot/data/Egypitan_dialogue.txt", encoding='utf-8'). read().strip().split('\n')
print(len(egypt_conv))
egypt_conv_pairs = []
for i in tqdm(range(len(egypt_conv))):
    if(i%2==0):
      s = []
      s.append( normalize_egypt(egypt_conv[i]))   # 0 2
      s.append( normalize_egypt(egypt_conv[i+1])) # 1 3
      egypt_conv_pairs.append(s)


In [ ]:
print(len(egypt_conv_pairs))
print(len(sents_egypt))
sents_egypt_all = egypt_conv_pairs
sents_egypt_all.extend( sents_egypt )
print(len(sents_egypt_all))

In [ ]:
MAX_LENGTH = 12
voc, pairs = loadPrepareData("", "", "")

In [ ]:
len(voc.word2count)

In [ ]:
1558/2

# trim

In [ ]:
MIN_COUNT = 1    # Minimum word count threshold for trimming

def trimRareWords(voc, pairs, MIN_COUNT):
    # Trim words used under the MIN_COUNT from the voc
    # el olelen msh bet7to f new list w n3ml BOW mn aol bl kotar
    # trim el VOCAB
    voc.trim(MIN_COUNT)
    
    # Filter out pairs with trimmed words
    # trim el PAIRS
    keep_pairs = []
    for pair in pairs:
        input_sentence = pair[0]
        output_sentence = pair[1]
        keep_input = True
        keep_output = True
        # Check input sentence
        for word in input_sentence.split(' '):
            if word not in voc.word2index:
                keep_input = False
                break
        # Check output sentence
        for word in output_sentence.split(' '):
            if word not in voc.word2index:
                keep_output = False
                break

        # Only keep pairs that do not contain trimmed word(s) in their input or output sentence
        if keep_input and keep_output:
            keep_pairs.append(pair)

    print("Trimmed from {} pairs to {}, {:.4f} of total".format(len(pairs), len(keep_pairs), len(keep_pairs) / len(pairs)))
    return keep_pairs


# Trim voc and pairs
pairs = trimRareWords(voc, pairs, MIN_COUNT)

In [ ]:
len(pairs) 

# batches

In [ ]:
import gensim
t_model = gensim.models.Word2Vec.load('C:/Users/AnyOneElse/GP_POP/Batot/data/models/full_grams_cbow_100_twitter.mdl')

In [ ]:
def cos_sim(u,v):
    numerator_ = u.dot(v)
    denominator_= np.sqrt(np.sum(np.square(u))) * np.sqrt(np.sum(np.square(v)))
    return numerator_/denominator_

In [ ]:
def compare(word):
    sim_max = -99999
    find_other = ""
    for key in voc.word2index:
        try:
            c1 = t_model.wv[ key ]

            c2 = t_model.wv[ word ]

            sim = cos_sim(c1,c2)
            if(sim > sim_max):
                find_other = key
                sim_max = sim

        except: 
            pass
    
    return find_other

In [ ]:
def indexesFromSentence(voc, sentence,print_):
    indeces = []
    new_sent = ""
    for word in sentence.split(' '):
        try:
            indeces.append( voc.word2index[word] )
            new_sent = new_sent + " " + word
        except:
            find_other = compare(word)
            print("compare")
            indeces.append( voc.word2index[find_other] )
            new_sent = new_sent + " " + find_other

    indeces.append(EOS_token)
    if(print_):
        print("filtered sent: " , new_sent)
    return indeces

###################################################################
###################################################################
def zeroPadding(l, fillvalue=PAD_token):
    return list(itertools.zip_longest(*l, fillvalue=fillvalue))

def binaryMatrix(l, value=PAD_token):
    m = []
    for i, seq in enumerate(l):
        m.append([])
        for token in seq:
            if token == PAD_token:
                m[i].append(0)
            else:
                m[i].append(1)
    return m

# Returns padded input sequence tensor and lengths
# sent -> indices -> padding
def inputVar(l, voc):
    indexes_batch = [indexesFromSentence(voc, sentence, False) for sentence in l] #sents-> indices
    lengths = torch.tensor([len(indexes) for indexes in indexes_batch]) # lens of indices
   
    padList = zeroPadding(indexes_batch) #esfar bm2as atol gomla
    padVar = torch.LongTensor(padList)   # torch to padded indices
    return padVar, lengths

# Returns padded target sequence tensor, padding mask, and max target length
def outputVar(l, voc):
    indexes_batch = [indexesFromSentence(voc, sentence, False) for sentence in l] #
    max_target_len = max([len(indexes) for indexes in indexes_batch])
    padList = zeroPadding(indexes_batch)
    mask = binaryMatrix(padList)
    mask = torch.BoolTensor(mask)
    padVar = torch.LongTensor(padList)
    return padVar, mask, max_target_len

# Returns all items for a given batch of pairs
def batch2TrainData(voc, pair_batch):
    
    pair_batch.sort(key=lambda x: len(x[0].split(" ")), reverse=True)
    input_batch, output_batch = [], []
    for pair in pair_batch:
        input_batch.append(pair[0])
        output_batch.append(pair[1])

    inp, lengths = inputVar(input_batch, voc)
    output, mask, max_target_len = outputVar(output_batch, voc)

    return inp, lengths, output, mask, max_target_len


# Example for validation
small_batch_size = 1
batches = batch2TrainData(voc, [random.choice(pairs) for _ in range(small_batch_size)])
input_variable, lengths, target_variable, mask, max_target_len = batches


*------------------------------------------------------------
*------------------------------------------------------------
*------------------------------------------------------------
*------------------------------------------------------------
*------------------------------------------------------------
*------------------------------------------------------------
*------------------------------------------------------------
*------------------------------------------------------------


# Define Models

## Encoder


In [ ]:
class EncoderRNN(nn.Module):
    def __init__(self, hidden_size, embedding, n_layers=1, dropout=0):
        super(EncoderRNN, self).__init__()
        self.n_layers = n_layers # 2
        self.hidden_size = hidden_size # 500
        self.embedding = embedding  # voc * 500
                                          
        self.gru = nn.GRU(hidden_size, hidden_size, n_layers,
                          dropout=(0 if n_layers == 1 else dropout), bidirectional=True)

    def forward(self, input_seq, input_lengths, hidden=None):
        # input_seq -> sentences in batch & input_lengths -> their lens
        #print(input_seq.shape) # 8*64 -> max_len * batches num
        #print(input_lengths.shape) #64->batches
        embedded = self.embedding(input_seq) # 8 , 64 , 500
        # Pack padded batch of sequences for RNN module
        packed = nn.utils.rnn.pack_padded_sequence(embedded, input_lengths)
        
        # Forward pass through GRU
        outputs, hidden = self.gru(packed, hidden)
        
        # Unpack padding
        outputs, _ = nn.utils.rnn.pad_packed_sequence(outputs)
        
        # Sum bidirectional GRU outputs
        outputs = outputs[:, :, :self.hidden_size] + outputs[:, : ,self.hidden_size:]
   
        # Return output and final hidden state
        return outputs, hidden

## Decoder


In [ ]:
# Luong attention layer
class Attn(nn.Module):
    def __init__(self, method, hidden_size):
        super(Attn, self).__init__()
        self.method = method
        self.hidden_size = hidden_size
        
    def dot_score(self, hidden, encoder_output):
        return torch.sum(hidden * encoder_output, dim=2)

    def forward(self, hidden, encoder_outputs):
        # Calculate the attention weights (energies) based on the given method
        if self.method == 'dot':
            attn_energies = self.dot_score(hidden, encoder_outputs)

        # Transpose max_length and batch_size dimensions
        attn_energies = attn_energies.t()

        # Return the softmax normalized probability scores (with added dimension)
        return F.softmax(attn_energies, dim=1).unsqueeze(1)

## seq2seq 

In [ ]:
class LuongAttnDecoderRNN(nn.Module):
    def __init__(self, attn_model, embedding, hidden_size, output_size, n_layers=1, dropout=0.1):
        super(LuongAttnDecoderRNN, self).__init__()

        # Keep for reference
        self.attn_model = attn_model
        self.hidden_size = hidden_size #500
        self.output_size = output_size
        self.n_layers = n_layers
        self.dropout = dropout

        # Define layers
        self.embedding = embedding
        self.embedding_dropout = nn.Dropout(dropout)
        self.gru = nn.GRU(hidden_size, hidden_size, n_layers, dropout=(0 if n_layers == 1 else dropout))
        self.concat = nn.Linear(hidden_size * 2, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)

        self.attn = Attn(attn_model, hidden_size)

    def forward(self, input_step, last_hidden, encoder_outputs):
        # Note: we run this one step (word) at a time
        # Get embedding of current input word
        embedded = self.embedding(input_step)
        embedded = self.embedding_dropout(embedded)
        # Forward through unidirectional GRU
        rnn_output, hidden = self.gru(embedded, last_hidden)
        # Calculate attention weights from the current GRU output
        attn_weights = self.attn(rnn_output, encoder_outputs)
        # Multiply attention weights to encoder outputs to get new "weighted sum" context vector
        context = attn_weights.bmm(encoder_outputs.transpose(0, 1))
        # Concatenate weighted context vector and GRU output using Luong eq. 5
        rnn_output = rnn_output.squeeze(0)
        context = context.squeeze(1)
        concat_input = torch.cat((rnn_output, context), 1)
        concat_output = torch.tanh(self.concat(concat_input))
        # Predict next word using Luong eq. 6
        output = self.out(concat_output)
        output = F.softmax(output, dim=1)
        # Return output and final hidden state
        return output, hidden

# loss

In [ ]:
def maskNLLLoss(inp, target, mask):
    nTotal = mask.sum()
    crossEntropy = -torch.log(torch.gather(inp, 1, target.view(-1, 1)).squeeze(1))
    loss = crossEntropy.masked_select(mask).mean()
    loss = loss.to(device)
    return loss, nTotal.item()

# train

In [ ]:
def train(input_variable, lengths, target_variable, mask, max_target_len, encoder, decoder, 
          embedding,
          encoder_optimizer, decoder_optimizer, batch_size, clip, max_length=MAX_LENGTH):

    ###################################################################################
    # init
    # Zero gradients
    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    # Set device options
    input_variable = input_variable.to(device)
    target_variable = target_variable.to(device)
    mask = mask.to(device)
    # Lengths for rnn packing should always be on the cpu
    lengths = lengths.to("cpu")

    # Initialize variables
    loss = 0
    print_losses = []
    n_totals = 0


    ########################################################################
    # encoder => Forward pass through encoder
    encoder_outputs, encoder_hidden = encoder(input_variable, lengths)

    
    # Create initial decoder input (start with SOS tokens for each sentence)
    decoder_input = torch.LongTensor([[SOS_token for _ in range(batch_size)]])
    decoder_input = decoder_input.to(device)

    # Set initial decoder hidden state to the encoder's final hidden state
    decoder_hidden = encoder_hidden[:decoder.n_layers]

    # Determine if we are using teacher forcing this iteration
    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

    # Forward batch of sequences through decoder one time step at a time
    if use_teacher_forcing:
        for t in range(max_target_len):
            decoder_output, decoder_hidden = decoder(
                              decoder_input, decoder_hidden, encoder_outputs
                          )
            # Teacher forcing: next input is current target
            decoder_input = target_variable[t].view(1, -1)
            # Calculate and accumulate loss
            mask_loss, nTotal = maskNLLLoss(decoder_output, target_variable[t], mask[t])
            loss += mask_loss
            print_losses.append(mask_loss.item() * nTotal)
            n_totals += nTotal
    else:
        for t in range(max_target_len):
            decoder_output, decoder_hidden = decoder(
                decoder_input, decoder_hidden, encoder_outputs
            )
            # No teacher forcing: next input is decoder's own current output
            _, topi = decoder_output.topk(1)
            decoder_input = torch.LongTensor([[topi[i][0] for i in range(batch_size)]])
            decoder_input = decoder_input.to(device)
            # Calculate and accumulate loss
            mask_loss, nTotal = maskNLLLoss(decoder_output, target_variable[t], mask[t])
            loss += mask_loss
            print_losses.append(mask_loss.item() * nTotal)
            n_totals += nTotal

    # Perform backpropatation
    loss.backward()

    # Clip gradients: gradients are modified in place
    _ = nn.utils.clip_grad_norm_(encoder.parameters(), clip)
    _ = nn.utils.clip_grad_norm_(decoder.parameters(), clip)

    # Adjust model weights
    encoder_optimizer.step()
    decoder_optimizer.step()

    return sum(print_losses) / n_totals

# iteration

In [ ]:
def trainIters(model_name, voc, pairs, encoder, 
               decoder, encoder_optimizer, decoder_optimizer,
               embedding, encoder_n_layers, decoder_n_layers,
               save_dir, n_iteration, batch_size, print_every,
               save_every, clip, corpus_name, loadFilename):

        # Load batches for each iteration
        # for each iteraion batch
    training_batches = [batch2TrainData(voc, [random.choice(pairs) for _ in range(batch_size)])
                      for _ in range(n_iteration)] 

    # Initializations
    print('Initializing ...')
    start_iteration = 1
    print_loss = 0
    if loadFilename:
        start_iteration = checkpoint['iteration'] + 1
        print("start at : " , start_iteration)

    # Training loop
    print("Training...")
    for iteration in range(start_iteration, n_iteration + 1):
        training_batch = training_batches[iteration - 1]
        # Extract fields from batch
        input_variable, lengths, target_variable, mask, max_target_len = training_batch

        # Run a training iteration with batch
        loss = train(input_variable, lengths, target_variable, mask, max_target_len, encoder,
                     decoder, embedding, encoder_optimizer, decoder_optimizer, batch_size, clip)
        print_loss += loss

        # Print progress
        if iteration % print_every == 0:
            print_loss_avg = print_loss / print_every
            print("Iteration: {}; Percent complete: {:.1f}%; Average loss: {:.4f}".format(iteration, iteration / n_iteration * 100, print_loss_avg))
            print_loss = 0

        # Save checkpoint
        if (iteration % save_every == 0):
            directory = os.path.join(save_dir, model_name, corpus_name, '{}-{}_{}'.format(encoder_n_layers, decoder_n_layers, hidden_size))
            print("save to:", directory)
            if not os.path.exists(directory):
                os.makedirs(directory)
            torch.save({
                'iteration': iteration,
                'en': encoder.state_dict(),
                'de': decoder.state_dict(),
                'en_opt': encoder_optimizer.state_dict(),
                'de_opt': decoder_optimizer.state_dict(),
                'loss': loss,
                'voc_dict': voc.__dict__,
                'embedding': embedding.state_dict()
            }, os.path.join(directory, '{}_{}.tar'.format(iteration, 'checkpoint')))

------------------------------------------------------------------

# init

In [ ]:
# Configure models
model_name = 'cb_model'
attn_model = 'dot'
hidden_size = 500
encoder_n_layers = 1
decoder_n_layers = 1
dropout = 0.1
batch_size = 1

save_dir = os.path.join("data", "save")

# Set checkpoint to load from; set to None if starting from scratch
loadFilename = None
checkpoint_iter = 1000
loadFilename = "C:/Users/AnyOneElse/GP_POP/Batot/data/models/save/cb_model/corpus_name/1-1_500/1499_checkpoint.tar"

# Load model if a loadFilename is provided
if loadFilename:
    # If loading on same machine the model was trained on
    #checkpoint = torch.load(loadFilename)
    # If loading a model trained on GPU to CPU
    checkpoint = torch.load(loadFilename, map_location=torch.device('cpu'))
    encoder_sd = checkpoint['en']
    decoder_sd = checkpoint['de']
    encoder_optimizer_sd = checkpoint['en_opt']
    decoder_optimizer_sd = checkpoint['de_opt']
    embedding_sd = checkpoint['embedding']
    #voc.__dict__ = checkpoint['voc_dict']


print('Building encoder and decoder ...')
# Initialize word embeddings
embedding = nn.Embedding(voc.num_words, hidden_size)
#if loadFilename:
#    embedding.load_state_dict(embedding_sd)
# Initialize encoder & decoder models
encoder = EncoderRNN(hidden_size, embedding, encoder_n_layers, dropout)
decoder = LuongAttnDecoderRNN(attn_model, embedding, hidden_size, voc.num_words, decoder_n_layers, dropout)
if loadFilename:
    encoder.load_state_dict(encoder_sd)
    decoder.load_state_dict(decoder_sd)
# Use appropriate device
encoder = encoder.to(device)
decoder = decoder.to(device)
print('Models built and ready to go!')

# train

In [ ]:
# Configure training/optimization
#%%time
clip = 50.0
teacher_forcing_ratio = 1.0
learning_rate = 0.0001
decoder_learning_ratio = 5.0
n_iteration = 1500
print_every = 100
save_every = n_iteration-1

# Ensure dropout layers are in train mode
encoder.train()
decoder.train()

# Initialize optimizers
print('Building optimizers ...')
encoder_optimizer = optim.Adam(encoder.parameters(), lr=learning_rate)
decoder_optimizer = optim.Adam(decoder.parameters(), lr=learning_rate * decoder_learning_ratio)
if loadFilename:
    encoder_optimizer.load_state_dict(encoder_optimizer_sd)
    decoder_optimizer.load_state_dict(decoder_optimizer_sd)


print("Starting Training!")
trainIters(model_name, voc, pairs, encoder, decoder, encoder_optimizer, decoder_optimizer,
           embedding, encoder_n_layers, decoder_n_layers, save_dir, n_iteration, batch_size,
           print_every, save_every, clip, "corpus_name", loadFilename)

In [ ]:
import numpy as np

# test

In [ ]:
class GreedySearchDecoder(nn.Module):
    def __init__(self, encoder, decoder):
        super(GreedySearchDecoder, self).__init__()
        self.encoder = encoder
        self.decoder = decoder

    def forward(self, input_seq, input_length, max_length):
        # Forward input through encoder model
        encoder_outputs, encoder_hidden = self.encoder(input_seq, input_length)
        # Prepare encoder's final hidden layer to be first hidden input to the decoder
        decoder_hidden = encoder_hidden[:decoder.n_layers]
        # Initialize decoder input with SOS_token
        decoder_input = torch.ones(1, 1, device=device, dtype=torch.long) * SOS_token
        # Initialize tensors to append decoded words to
        all_tokens = torch.zeros([0], device=device, dtype=torch.long)
        all_scores = torch.zeros([0], device=device)
        # Iteratively decode one word token at a time
        for _ in range(max_length):
            # Forward pass through decoder
            decoder_output, decoder_hidden = self.decoder(decoder_input, decoder_hidden, encoder_outputs)
            # Obtain most likely word token and its softmax score
            decoder_scores, decoder_input = torch.max(decoder_output, dim=1)
            # Record token and score
            all_tokens = torch.cat((all_tokens, decoder_input), dim=0)
            all_scores = torch.cat((all_scores, decoder_scores), dim=0)
            # Prepare current token to be next decoder input (add a dimension)
            decoder_input = torch.unsqueeze(decoder_input, 0)
        # Return collections of word tokens and scores
        return all_tokens, all_scores

In [ ]:
from mtranslate import translate
from emotions import all_emotions
from sleep_check import sleep_read
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow as tf
keras = tf.keras
#
model_path = 'C:/Users/AnyOneElse/GP_POP/Batot/data/models/USE_binary_model.h5'
binary_model = tf.keras.models.load_model(model_path , custom_objects={'KerasLayer': hub.KerasLayer})


In [55]:
from mtranslate import translate
MAX_LENGTH = 12

def evaluate(encoder, decoder, searcher, voc, sentence, max_length=MAX_LENGTH):
    ### Format input sentence as a batch
    # words -> indexes
    indexes_batch = [indexesFromSentence(voc, sentence, True)]
    # Create lengths tensor
    lengths = torch.tensor([len(indexes) for indexes in indexes_batch])
    # Transpose dimensions of batch to match models' expectations
    input_batch = torch.LongTensor(indexes_batch).transpose(0, 1)
    # Use appropriate device
    input_batch = input_batch.to(device)
    lengths = lengths.to("cpu")
    # Decode sentence with searcher
    tokens, scores = searcher(input_batch, lengths, max_length)
    # indexes -> words
    decoded_words = [voc.index2word[token.item()] for token in tokens]
    return decoded_words


############################################################
############################################################
############################################################
def evaluateInput():
    old_mood = ""
    iter = 0
    five_sent_list = []
    while(1):
        
        input_sentence = speech2txt().replace("." , " ")
        #input_sentence = input().replace("." , " ")
        if(input_sentence != '0'):
            five_sent_list.append(input_sentence)

        #**********************************************
        # binary model  
        if(len(five_sent_list) >= 3):
            print("***************check mode *5555555555555")
            sent = ""
            for i in range(len(five_sent_list)):
                sent = sent + " " + five_sent_list[i].replace("\n","")

            all_sent_eng = translate(sent)
            print("five sentences:  " ,  all_sent_eng)
            result = binary_model.predict([all_sent_eng])[0][0]
            five_sent_list = []
            print(result)
            if(result > 0.5):
                speak_ar("أَنْتَ اتحسنت هتركك تُرَكِّزُ فى السواقة ")
                file_batot = open("C:/Users/AnyOneElse/GP_POP/Batot/data/file_chat_user_history.txt", "a" , encoding='utf-8')
                file_batot.write("باى يا حبيب هارتى هيوحشنى الكلام معاك" + "\n")
                file_batot.write("end" + "\n")
                file_batot.close()

                break
        #**********************************************
        file = open("C:/Users/AnyOneElse/GP_POP/Batot/data/file_chat_user_history.txt", "a" , encoding='utf-8')
        file.write(input_sentence + "\n")
        file.close()
   
        if(input_sentence == '0'):
            speak_ar("مش سامع")
            
        else:
            input_sentence = normalize_egypt(  input_sentence  ).replace("اوي", "").replace("جدا", "").replace("خالص", "").replace("انا", "").replace("انتا", "")
            # Check if it is quit case
            print("translated:  ", translate(input_sentence))
            if ( (translate(input_sentence).find('bye') != -1) or (translate(input_sentence).find('باي') != -1)) or (translate(input_sentence).find('WL') != -1) :
                
                file_end = open("C:/Users/AnyOneElse/GP_POP/Batot/data/file_chat_user_history.txt", "a" )
                file_end.write("end"+"\n")
                file_end.close()
            
                file_batot = open("C:/Users/AnyOneElse/GP_POP/Batot/data/file_chat_batot_history.txt", "a" , encoding='utf-8')
                file_batot.write("باى يا حبيب هارتى هيوحشنى الكلام معاك" + "\n")
                file_batot.write("end" + "\n")
                file_batot.close()
                
                break

            input_sentence = normalizeString(input_sentence)
            print("norm input: " , input_sentence)

            # Evaluate sentence
            output_words = evaluate(encoder, decoder, searcher, voc, input_sentence)
            # Format and print response sentence
            output_words[:] = [x for x in output_words if not (x == 'EOS' or x == 'PAD')]

            print("***********************************************************")
            print("Cam_text Mood")
            file = open("C:/Users/AnyOneElse/GP_POP/cam_emotion_detect/mix_mood.txt", "r")
            cam_moods = file.readlines()
            file.close()
            mood_now = cam_moods[len(cam_moods) - 1 ].replace("\n", "")
            if((mood_now != old_mood) and (iter >= 1 )):
                print(mood_now)
                old_mood = mood_now
                all_emotions(mood_now)
                iter = 0
            else:
                iter = iter + 1
            
            print("iter num: ",iter)
                
            print("***********************************************************")
            print('Bottttttttttttttttttttttttt  :  ', ' '.join(output_words))
            speak_ar(' '.join(output_words))
            
            file_batot = open("C:/Users/AnyOneElse/GP_POP/Batot/data/file_chat_batot_history.txt", "a" , encoding='utf-8')
            file_batot.write(' '.join(output_words) + "\n")
            file_batot.close()
            

In [56]:
# Set dropout layers to eval mode
encoder.eval()
decoder.eval()
# Initialize search module
searcher = GreedySearchDecoder(encoder, decoder)

# Begin chatting (uncomment and run the following line to begin)
#evaluateInput()

Say something...please
Recognized: أنا بحبك
translated:   with ur love
norm input:  بحبك
filtered sent:   بحبك
***********************************************************
Cam_text Mood
iter num:  1
***********************************************************
Bottttttttttttttttttttttttt  :   و انا كمان
Say something...please
Recognized: أنا بحب كل الناس.
translated:   I love all people
norm input:  بحب كل الناس
filtered sent:   بحب كل الناس
***********************************************************
Cam_text Mood
hide
MOOOOOOOOOOOOOOOOOOOOOOOODDDDDDDDDDDD
hide
iter num:  0
***********************************************************
Bottttttttttttttttttttttttt  :   عشان انت روحك حلوه
Say something...please
Recognized: أنا بحبك
***************check mode *5555555555555
five sentences:   I love you I love all people I love you
0.56943405
